## Merge of the dataset of oscar and movies budet/popu/rating 

the_oscar_award.csv comes from:  https://www.kaggle.com/datasets/unanimad/the-oscar-award
movie_data_tmbd.csv comes from : https://www.kaggle.com/datasets/kakarlaramcharan/tmdb-data-0920

In [2]:
import pandas as pd

In [33]:
# oscars dataset : drop useless columns, change name to match the other datasets, keep only 4 categories of nominations
oscars = pd.read_csv('data/the_oscar_award.csv')
oscars = oscars[oscars['category'].str.contains('ACTOR|ACTOR IN A SUPPORTING ROLE|ACTRESS|ACTRESS IN A SUPPORTING ROLE|DIRECTING', regex=True)]
oscars.rename(columns={'film': 'movie_name'}, inplace=True)
oscars.rename(columns={'year_film': 'Movie release date'}, inplace=True)

oscars['movie_name'] = oscars['movie_name'].str.lower().str.replace(' ', '_')

oscars

,Movie release date,year_ceremony,ceremony,category,name,movie_name,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,the_noose,False
1,1927,1928,1,ACTOR,Emil Jannings,the_last_command,True
2,1927,1928,1,ACTRESS,Louise Dresser,a_ship_comes_in,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th_heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,sadie_thompson,False
...,...,...,...,...,...,...,...
10800,2023,2024,96,DIRECTING,Justine Triet,anatomy_of_a_fall,False
10801,2023,2024,96,DIRECTING,Martin Scorsese,killers_of_the_flower_moon,False
10802,2023,2024,96,DIRECTING,Christopher Nolan,oppenheimer,True
10803,2023,2024,96,DIRECTING,Yorgos Lanthimos,poor_things,False


In [34]:
# need to have only one row per movie: addition of a 'num_nominations' column to count the number of nominations per movie, 
# and a 'winner' column to know if the movie won the category and by who --> will be matched with the actors we are interested in

oscars_to_merge = oscars.groupby(['movie_name'], as_index=False).agg({
    'Movie release date': 'first',
    'category': lambda x: ', '.join(f"{cat}: {name}" for cat, name in zip(x, oscars.loc[x.index, 'name'])),
    'winner': lambda x: ', '.join(oscars.loc[x.index, 'name'][oscars.loc[x.index, 'winner']]),
    'category': 'count'
}).rename(columns={'category': 'num_nominations'})

oscars_to_merge

,movie_name,Movie release date,num_nominations,winner
0,'round_midnight,1986,1,
1,...and_justice_for_all,1979,1,
2,127_hours,2010,1,
3,12_angry_men,1957,1,
4,12_monkeys,1995,1,
...,...,...,...,...
1298,you_can't_take_it_with_you,1938,2,Frank Capra
1299,you_can_count_on_me,2000,1,
1300,z,1969,1,
1301,zero_dark_thirty,2012,1,


In [35]:
# tmdb dataset : drop useless columns, change name to match the other datasets

data_tmdb = pd.read_csv('data/movie_data_tmbd.csv', sep='|')
data_tmdb.drop(['adult', 'id', 'imdb_id', 'backdrop_path', 'belongs_to_collection', 'genres', 'homepage', 'original_language', 'original_title', 'overview', 'runtime', 'spoken_languages', 'video', 'poster_path', 'production_companies', 'production_countries', 'tagline', 'cast'], axis=1, inplace=True)

data_tmdb.rename(columns={'revenue': 'revenue_tmdb'}, inplace=True)
data_tmdb.rename(columns={'title': 'movie_name'}, inplace=True)
data_tmdb.rename(columns={'release_date': 'Movie release date'}, inplace=True)

data_tmdb['movie_name'] = data_tmdb['movie_name'].str.lower().str.replace(' ', '_')


data_tmdb

,budget,popularity,Movie release date,revenue_tmdb,status,movie_name,vote_average,vote_count,directors
0,0,3.892,1962-05-23,0,Released,the_elusive_corporal,5.9,10.0,"[{'credit_id': '52fe4626c3a36847f80ef68b', 'de..."
1,0,5.482,1962-11-12,0,Released,sundays_and_cybele,7.4,28.0,"[{'credit_id': '52fe4626c3a36847f80ef6c7', 'de..."
2,0,8.262,1962-05-24,0,Released,lonely_are_the_brave,7.5,70.0,"[{'credit_id': '52fe4626c3a36847f80ef733', 'de..."
3,0,7.83,1975-03-12,0,Released,f_for_fake,7.5,178.0,"[{'credit_id': '52fe4626c3a36847f80ef75b', 'de..."
4,500000,5.694,1962-10-09,0,Released,long_day's_journey_into_night,6.9,32.0,"[{'credit_id': '52fe4626c3a36847f80ef791', 'de..."
...,...,...,...,...,...,...,...,...,...
119933,17500,0.6000000000000001,2017-05-16,0,Released,the_enemies,8.5,2.0,[]
119934,50000,0.6000000000000001,NaN,0,Released,the_last_rodriguez,10.0,1.0,[]
119935,11000,3.142,2020-03-31,0,Released,space,5.6,5.0,"[{'credit_id': '5e84ad63e38bd80016abeacb', 'de..."
119936,0,3.279,2020-02-14,0,Released,room_441,2.0,1.0,"[{'credit_id': '5e8213810bc5290016f1a0fd', 'de..."


In [36]:
# CMU Movie dataset : kept all columns, change name to match the other datasets

original_data = pd.read_csv('data/movie.metadata.tsv', sep='\t', names= ['Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres'])
original_data.rename(columns={'Movie name': 'movie_name'}, inplace=True)

original_data['movie_name'] = original_data['movie_name'].str.lower().str.replace(' ', '_')

original_data[['movie_name', 'Movie release date']]


,movie_name,Movie release date
0,ghosts_of_mars,2001-08-24
1,getting_away_with_murder:_the_jonbenét_ramsey_...,2000-02-16
2,brun_bitter,1988
3,white_of_the_eye,1987
4,a_woman_in_flames,1983
...,...,...
81736,mermaids:_the_body_found,2011-03-19
81737,knuckle,2011-01-21
81738,another_nice_mess,1972-09-22
81739,the_super_dimension_fortress_macross_ii:_lover...,1992-05-21


In [44]:
# merge the 3 datasets always on the original_data dataset rows


# merge of original_data and data_tmdb
merged_data = original_data.merge(data_tmdb, on=['movie_name', 'Movie release date'], how='left')
merged_data


#merge of merged_data and oscars_to_merge

# Ensure that 'Movie release date' is in a valid datetime format.
def convert_to_datetime(date):
    # Check if the date is just a year (4 digits) and convert it to 'yyyy-01-01' format
    if len(str(date)) == 4 and str(date).isdigit():  # Only year (e.g., 1988)
        return pd.to_datetime(str(date) + '-01-01')
    else:
        return pd.to_datetime(date, errors='coerce')  # Convert if it's in a full date format

# Apply the conversion to the 'Movie release date' column
merged_data['Movie release date'] = merged_data['Movie release date'].apply(convert_to_datetime)

# Now extract the year from 'Movie release date' safely
merged_data['Movie release date'] = merged_data['Movie release date'].dt.year

final_merged_data = merged_data.merge(oscars_to_merge, on=['movie_name', 'Movie release date'], how='left')
final_merged_data





,Wikipedia movie ID,Freebase movie ID,movie_name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,budget,popularity,revenue_tmdb,status,vote_average,vote_count,directors,num_nominations,winner
0,975900,/m/03vyhn,ghosts_of_mars,2001.0,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",28000000,14.721,14010832,Released,5.1,657.0,"[{'credit_id': '52fe43089251416c75000d3d', 'de...",NaN,NaN
1,3196793,/m/08yl5d,getting_away_with_murder:_the_jonbenét_ramsey_...,2000.0,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28463795,/m/0crgdbh,brun_bitter,1988.0,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9363483,/m/0285_cd,white_of_the_eye,1987.0,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,261236,/m/01mrr1,a_woman_in_flames,1983.0,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81737,35228177,/m/0j7hxnt,mermaids:_the_body_found,2011.0,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81738,34980460,/m/0g4pl34,knuckle,2011.0,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81739,9971909,/m/02pygw1,another_nice_mess,1972.0,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81740,913762,/m/03pcrp,the_super_dimension_fortress_macross_ii:_lover...,1992.0,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
